In [ ]:
from bs4 import BeautifulSoup
import requests
!pip install Levenshtein
from Levenshtein import ratio
import pandas as pd
import time
from google.colab import drive
import json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 30.9 MB/s eta 0:00:00


Подключение Google Drive для дальнейшей загрузки итогового файла

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sites = 'https://www.mdpi.com/1996-1073/11/' ##vol 11 (2017) iss. 1

>**Сбор данных с сайта (список авторов и цитируемых)**

In [ ]:
graph_ref = []
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36', }
count_summary = 0
authors_list = set()
citir_list = set()

for i in range(1, 2):
    site = sites + str(i)
    resp = requests.get(site, headers=headers).text
    data = BeautifulSoup(resp, 'html.parser')
    count = data.find('div', class_='content__container content__container--overflow-initial')
    count = int(count.find('h1').text.split()[-2])
    print(count_summary, count, count + count_summary)

    start_time = time.time()

    for x in range(count_summary, count_summary + count):
        site_current = site + '/' + str(x + 1)
        resp_current = requests.get(site_current, headers=headers).text
        data_current = BeautifulSoup(resp_current, 'html.parser') #сайт с одной статьей
        if data_current == None:
          continue
        a = str()
        list_authors_data = []
        authors_data = data_current.find('div', class_='art-authors hypothesis_container') #берем информацию об авторах
        if authors_data == None:
            continue
        for name in authors_data.find_all('div', class_='profile-card-drop'):
            list_authors_data.append(name.text.replace("\n", '').replace(',', '').strip())
        for name in authors_data.find_all('div', class_='profile-card-drop'):
            authors_list.add(name.text.replace("\n", '').replace(',', '').strip())
        if data_current.find('section', id='html-references_list') != None:
            references_data = data_current.find('section', id='html-references_list').find('ol', class_='html-xx')
        if references_data == None:
            references_data = data_current.find('section', id='html-references_list').find('ol', class_='html-xxx')
        u = 0
        list_keywords = [] # берем информацию о ключевых словах
        keywords_data = data_current.find('div', class_='html-gwd-group')
        if keywords_data != None:
            for keyw in keywords_data.find_all('a'):
                keyw_txt = keyw.text
                list_keywords.append(keyw_txt)
        year = 0
        year_data = data_current.find('div', class_='pubhistory') # берем информацию о годе выхода статьи
        if year_data == None:
            continue
        for yer in year_data.find_all('span'):
            yer_txt = yer.text
            if 'Published:' in yer_txt.split():
                year = int(yer_txt.split()[-1])
        name_data = data_current.find('h1', class_='title hypothesis_container') #берем информацию о названии статьи
        name_stat = name_data.text.strip()
        if references_data == None:
            continue
        for ref in references_data.find_all('li', class_='html-x'): # берем информацию о цитируемых
            ref1 = str(ref)
            s = str()
            zr = []
            list_citir_data = []
            if ref1.find('<', int(ref1.find('<')) + 1) - ref1.find('>') != 1:
                s = ref1[ref1.find('>') + 1:ref1.find('<', int(ref1.find('<')) + 1)]
                zr = list(map(str, s.replace(',', '').split(';')))
            for kl in range(len(zr) - 1):
                citir_list.add(zr[kl])
                list_citir_data.append(zr[kl])
            if len(zr) > 0:
                stat_cit_data = zr[-1] # берем информацию о названии цитируемой статьи
                name_stat_cit = stat_cit_data[stat_cit_data.find('.') + 1:]
                if len(list(name_stat_cit.split())) == 0:
                    break
                else:
                    if list(name_stat_cit.split())[0][-1] == '.':
                        tkk = 0
                        while list(name_stat_cit.split())[0][-1] == '.':
                            name_stat_cit = name_stat_cit[name_stat_cit.find('.') + 1:]
                            if len(list(name_stat_cit.split())) == 0:
                                tkk = 1
                                break
                        if tkk == 1:
                            break
                        name_stat_cit = name_stat_cit.strip()
            for i in range(len(list_authors_data)):
                for j in range(len(list_citir_data)):
                    graph_ref.append([list_authors_data[i], list_citir_data[j], year, list_keywords, name_stat, name_stat_cit, site_current]) # добавляем все в рабочий массив
            u += 1
            if u > 1000: #кол-во цитируемых в пунктах (меняется)
                break
        if x % 100 == 0: print(x, -1)
        if x > 100: break #кол-во статьей (меняется)
        if x % 20 == 0: print(x)
    print('Done')

    end_time = time.time()
    print(end_time - start_time)
    print()
    count_summary += count

authors_list = list(authors_list)
citir_list = list(citir_list)

0 257 257
0 -1
0
20
40
60
80
100 -1
100
Done
80.26178932189941




>**Сопоставление полного ФИО и сокращенного**

In [ ]:
start_time = time.time()

d = {}
print(len(citir_list))
for i in range(len(citir_list)):
    maxx = 0
    correct = ''
    for j in range(len(authors_list)):
        f = ratio(authors_list[j], citir_list[i])
        if f > maxx and f >= 0.48:
            maxx = f
            correct = authors_list[j]
    if correct != '':
        for sec_name in citir_list[i].split():
            if sec_name[-1] != '.':
                tr = 0
                for lkip in correct.split():
                    if lkip == sec_name:
                        jkl = 0
                        for tyu in citir_list[i].split():
                            if tyu != sec_name:
                                for jui in correct.split():
                                    if jui != lkip:
                                        if tyu[0] in jui:
                                            jkl += 1
                        if jkl != 0:
                            tr += 1
                if tr != 0:
                    d[citir_list[i]] = [correct, maxx]
    if i % 100 == 0:
      print(i)

end_time = time.time()
print("Done!")
print()
print(end_time - start_time)

1529
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
Done!

0.5153694152832031


>**Составление конечного графа цитирований (цитируемые заменяются их полным ФИО)**

In [ ]:
graph_ref_result = []
graph_ref_ost = []
for i in range(len(graph_ref)):
    if graph_ref[i][1] in d:
        # graph_ref_result = [автор статьи, цитируемый автор, год статьи, ключевые слова, название статьи, название цитируемой статьи, ссылка на статью, (# кол-во цитирований автора статьи цитируемого автора, кол-во цитирований названия статьи]
        graph_ref_result.append([graph_ref[i][0], d[graph_ref[i][1]][0], graph_ref[i][2], graph_ref[i][3], graph_ref[i][4], graph_ref[i][5].strip(), graph_ref[i][6]])
        graph_ref_ost.append([graph_ref[i][0], graph_ref[i][1]])


In [ ]:
print(len(graph_ref_result))
start_time = time.time()
for i in range(len(graph_ref_result)):
    auth = graph_ref_result[i][0]
    citirr = graph_ref_result[i][1]
    sch = 0
    name_stat_main = graph_ref_result[i][4]
    sch1 = 0
    for j in range(len(graph_ref_result)):
        if graph_ref_result[j][0] == auth and graph_ref_result[j][1] == citirr:
            sch += 1
        if graph_ref_result[j][5] == name_stat_main:
            sch1 += 1
    graph_ref_result[i].append(sch)
    graph_ref_result[i].append(sch1)
    if i % 1000 == 0:
        print(i)
print('Done!')
end_time = time.time()
print(end_time - start_time)

312
0
Done!
0.05365395545959473


In [ ]:
# ВСТАВИТЬ НОМЕР ПРОВЕРЯЕМОГО НОМЕРА ЭЛЕМЕНТА В МАССИВЕ ГРАФА ("ГРЯЗНОГО" И "ЧИСТОГО")
op = 200
print(graph_ref_ost[op])
print(graph_ref_result[op])

['Ali Selim', ' Mekhilef S.']
['Ali Selim', 'Saad Mekhilef', 2022, ['multilevel inverter (MLI)', 'reduced component count', 'asymmetrical', 'pulse width modulation (PWM)', 'photovoltaic (PV) systems', 'electric vehicles (EV)']]


Создание датафрейма из полученных данных

In [ ]:
column_author = []
column_citir = []
column_year = []
column_keyw = []
column_sch = []
column_stat_name = []
column_por_num = [] # порядковый номер автора внутри статьи
column_cit_cnt = []
column_link = []

In [ ]:
# таблица 2 (обновленная)
for i in range(len(graph_ref_result)):
    if len(graph_ref_result[i]) > 8: # поменяно с 9
        for j in range(len(graph_ref_result[i][3])):
            column_stat_name.append(graph_ref_result[i][4])
            column_author.append(graph_ref_result[i][0])
            column_keyw.append(graph_ref_result[i][3][j])
            column_year.append(graph_ref_result[i][2])
            column_link.append(graph_ref_result[i][6])

In [ ]:
df_dict = {'name_stat': column_stat_name, 'author': column_author, 'year': column_year, 'keyword': column_keyw, 'link': column_link}
df = pd.DataFrame(df_dict)
df

,name_stat,author,year,keyword,link
0,A Parallel Restoration for Black Start of Micr...,Jing Wang,2017,microgrids,https://www.mdpi.com/1996-1073/11/1/1
1,A Parallel Restoration for Black Start of Micr...,Jing Wang,2017,black-start,https://www.mdpi.com/1996-1073/11/1/1
2,A Parallel Restoration for Black Start of Micr...,Jing Wang,2017,characteristics of distributed generations (DGs),https://www.mdpi.com/1996-1073/11/1/1
3,A Parallel Restoration for Black Start of Micr...,Jing Wang,2017,variation coefficient method,https://www.mdpi.com/1996-1073/11/1/1
4,A Parallel Restoration for Black Start of Micr...,Jing Wang,2017,parallel restoration,https://www.mdpi.com/1996-1073/11/1/1
...,...,...,...,...,...
1845,A Practical Approach to Localize Simultaneous ...,Kyeong-Hwa Kim,2018,free-wheeling mode detection,https://www.mdpi.com/1996-1073/11/1/101
1846,A Practical Approach to Localize Simultaneous ...,Kyeong-Hwa Kim,2018,PWM inverter,https://www.mdpi.com/1996-1073/11/1/101
1847,A Practical Approach to Localize Simultaneous ...,Kyeong-Hwa Kim,2018,reliable on-line fault localization,https://www.mdpi.com/1996-1073/11/1/101
1848,A Practical Approach to Localize Simultaneous ...,Kyeong-Hwa Kim,2018,seven fault groups,https://www.mdpi.com/1996-1073/11/1/101


Запись полученной таблицы в excel таблицу

In [ ]:
df.to_excel('/content/drive/MyDrive/Project/tabl2.xlsx')